In [100]:
import os

os.getcwd()

from pathlib import Path

# Project root = two levels up from notebooks/EDA
PROJECT_ROOT = Path.cwd().parents[1]

DATA_RAW = PROJECT_ROOT / "data" / "raw"

DATA_RAW

list(DATA_RAW.iterdir())


[PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/speeds_monthly_2015.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/journeyperformance_monthly_2017.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/bus_lane_geometry.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/able_ace_start.csv'),
 PosixPath('/Users/danielbrown/Desktop/mta-ace-buses/data/raw/waitassessment_monthly_2015.csv')]

In [101]:
import pandas as pd

df_speeds = pd.read_csv(DATA_RAW / "journeyperformance_monthly_2017.csv")
df_speeds.head()


,month,borough,trip_type,route_id,period,number_of_customers,additional_bus_stop_time,additional_travel_time,customer_journey_time_performance
0,2017-08-01,Bronx,LCL/LTD,BX1,Off-Peak,"219,531.64",2.051653,0.696010,70.1999545%
1,2017-08-01,Bronx,LCL/LTD,BX1,Peak,"144,822.4",1.963801,0.855400,68.3975101%
2,2017-08-01,Bronx,LCL/LTD,BX10,Off-Peak,"142,222.2",1.262757,0.150511,78.2773277%
3,2017-08-01,Bronx,LCL/LTD,BX10,Peak,"98,257.17",0.869035,-0.195989,85.6885107%
4,2017-08-01,Bronx,LCL/LTD,BX11,Off-Peak,"177,312.86",1.555327,0.228310,77.2575616%


In [102]:
#df_speeds = df_speeds[df_speeds["period"] == "Off-Peak"]

df_speeds["customer_journey_time_performance"] = (
    df_speeds["customer_journey_time_performance"]
        .str.replace("%", "", regex=False)
        .astype(float)
)

avg_perf = (
    df_speeds
        .groupby(["month", "route_id"], as_index=False)
        ["customer_journey_time_performance"]
        .mean()
)

df_grouped = avg_perf

In [103]:
#Dealing with ACE

In [104]:
import pandas as pd

df_ace = pd.read_csv(DATA_RAW / "able_ace_start.csv")
df_ace.head()

import pandas as pd
from datetime import datetime

df_ace_filtered = df_ace

# Convert Implementation Date to datetime
df_ace_filtered["Implementation Date"] = pd.to_datetime(
    df_ace_filtered["Implementation Date"], format="%m/%d/%Y"
)

# Function to snap to closest first of month
def snap_to_nearest_first_of_month(dt: pd.Timestamp) -> pd.Timestamp:
    first_prev = pd.Timestamp(year=dt.year, month=dt.month, day=1)
    # first of next month
    if dt.month == 12:
        first_next = pd.Timestamp(year=dt.year + 1, month=1, day=1)
    else:
        first_next = pd.Timestamp(year=dt.year, month=dt.month + 1, day=1)
    
    # return whichever is closer
    if (dt - first_prev) <= (first_next - dt):
        return first_prev
    else:
        return first_next

# Apply snapping
df_ace_filtered["Implementation FirstOfMonth"] = df_ace_filtered["Implementation Date"].apply(snap_to_nearest_first_of_month)

# Show result
df_ace_filtered.head()

,Route,Program,Implementation Date,Implementation FirstOfMonth
0,M15+,ABLE,2019-10-07,2019-10-01
1,B44+,ABLE,2019-10-30,2019-11-01
2,M14+,ABLE,2019-11-21,2019-12-01
3,B46+,ABLE,2020-02-19,2020-03-01
4,M23+,ABLE,2020-08-10,2020-08-01


In [105]:
df_grouped["month"] = pd.to_datetime(df_grouped["month"])
df_ace_filtered["Implementation FirstOfMonth"] = pd.to_datetime(
    df_ace_filtered["Implementation FirstOfMonth"]
)

df_ace_filtered = df_ace_filtered.rename(columns={"Route": "route_id"})

program_timeline = (
    df_ace_filtered
    .pivot_table(
        index="route_id",
        columns="Program",
        values="Implementation FirstOfMonth",
        aggfunc="min"
    )
    .reset_index()
)

program_timeline.head()

Program,route_id,ABLE,ACE
0,B11,NaT,2025-11-01
1,B25,2022-12-01,2024-10-01
2,B26,2023-10-01,2024-10-01
3,B35,NaT,2024-09-01
4,B41,NaT,2024-09-01


In [106]:
df = df_grouped.merge(
    program_timeline,
    on="route_id",
    how="inner"
)
df.head()

,month,route_id,customer_journey_time_performance,ABLE,ACE
0,2017-08-01,B11,80.690388,NaT,2025-11-01
1,2017-08-01,B25,70.831597,2022-12-01,2024-10-01
2,2017-08-01,B26,76.949265,2023-10-01,2024-10-01
3,2017-08-01,B35,66.809800,NaT,2024-09-01
4,2017-08-01,B41,69.176290,NaT,2024-09-01


In [107]:
#Prophet Starts

In [108]:
# Rename columns to Prophet's required schema
df = df.rename(columns={
    "month": "ds",
    "customer_journey_time_performance": "y"
})

# Ensure ds is datetime
df["ds"] = pd.to_datetime(df["ds"])
df

,ds,route_id,y,ABLE,ACE
0,2017-08-01,B11,80.690388,NaT,2025-11-01
1,2017-08-01,B25,70.831597,2022-12-01,2024-10-01
2,2017-08-01,B26,76.949265,2023-10-01,2024-10-01
3,2017-08-01,B35,66.809800,NaT,2024-09-01
4,2017-08-01,B41,69.176290,NaT,2024-09-01
...,...,...,...,...,...
5042,2025-12-01,Q58,59.487712,2023-07-01,2024-07-01
5043,2025-12-01,Q6,55.250970,NaT,2025-09-01
5044,2025-12-01,Q69,81.482458,NaT,2024-10-01
5045,2025-12-01,S46,72.187917,NaT,2024-09-01


In [109]:
from prophet import Prophet
import pandas as pd
import numpy as np

# ------------------------------
# CONFIG
# ------------------------------
MIN_OBS = 12

COVID_START = "2020-03-01"
COVID_END   = "2021-03-01"   # conservative: include recovery

# ------------------------------
# Step 0: Prepare results list
# ------------------------------
results = []

# ------------------------------
# Step 1: Loop over routes
# ------------------------------
for route in df["route_id"].unique():

    # Select route data
    df_r = df[df["route_id"] == route].sort_values("ds").copy()

    # ------------------------------
    # Step 1a: REMOVE COVID PERIOD
    # ------------------------------
    df_r = df_r[
        ~((df_r["ds"] >= COVID_START) & (df_r["ds"] <= COVID_END))
    ]

    # Skip if too little data remains
    if len(df_r) < MIN_OBS:
        continue

    # ------------------------------
    # Step 2: Policy indicators
    # ------------------------------
    df_r["is_able"] = (
        df_r["ABLE"].notna() & (df_r["ds"] >= df_r["ABLE"])
    ).astype(int)

    df_r["is_ace"] = (
        df_r["ACE"].notna() & (df_r["ds"] >= df_r["ACE"])
    ).astype(int)

    # ------------------------------
    # Step 3: Fit Prophet
    # ------------------------------
    try:
        m0 = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.05  # conservative trend
        )

        m0.add_regressor("is_able")
        m0.add_regressor("is_ace")

        m0.fit(df_r[["ds", "y", "is_able", "is_ace"]])

    except Exception as e:
        print(f"Skipping route {route}: {e}")
        continue

    # ------------------------------
    # Step 4: Counterfactual (no ABLE / ACE)
    # ------------------------------
    future = df_r[["ds"]].copy()
    future["is_able"] = 0
    future["is_ace"] = 0

    forecast = m0.predict(future)

    df_r = df_r.merge(
        forecast[["ds", "yhat"]],
        on="ds",
        how="left"
    )

    df_r.rename(columns={"yhat": "yhat_no_policy"}, inplace=True)

    # ------------------------------
    # Step 5: Effect masks
    # ------------------------------
    able_only_mask = (df_r["is_able"] == 1) & (df_r["is_ace"] == 0)
    ace_mask = (df_r["is_ace"] == 1)

    # ------------------------------
    # Step 6: Compute effects
    # ------------------------------
    able_effect = (
        (df_r.loc[able_only_mask, "y"].mean()
         - df_r.loc[able_only_mask, "yhat_no_policy"].mean())
        / df_r.loc[able_only_mask, "yhat_no_policy"].mean()
    ) * 100 if able_only_mask.any() else np.nan

    ace_effect = (
        (df_r.loc[ace_mask, "y"].mean()
         - df_r.loc[ace_mask, "yhat_no_policy"].mean())
        / df_r.loc[ace_mask, "yhat_no_policy"].mean()
    ) * 100 if ace_mask.any() else np.nan

    # ------------------------------
    # Step 7: Store results
    # ------------------------------
    results.append({
        "route_id": route,
        "able_effect_pct": able_effect,
        "ace_incremental_effect_pct": ace_effect
    })

# ------------------------------
# Step 8: Final dataframe
# ------------------------------
df_effects = pd.DataFrame(results)

# ------------------------------
# Step 9: Total compounded effect
# ------------------------------
df_effects["total_effect_pct"] = (
    (1 + df_effects["able_effect_pct"].fillna(0) / 100)
    * (1 + df_effects["ace_incremental_effect_pct"].fillna(0) / 100)
    - 1
) * 100

DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

df_effects.to_csv(
    DATA_PROCESSED / "journeyperformance_overall.csv",
    index=False
)


df_effects



10:37:24 - cmdstanpy - INFO - Chain [1] start processing
10:37:25 - cmdstanpy - INFO - Chain [1] done processing
10:37:25 - cmdstanpy - INFO - Chain [1] start processing
10:37:25 - cmdstanpy - INFO - Chain [1] done processing
10:37:25 - cmdstanpy - INFO - Chain [1] start processing
10:37:25 - cmdstanpy - INFO - Chain [1] done processing
10:37:25 - cmdstanpy - INFO - Chain [1] start processing
10:37:26 - cmdstanpy - INFO - Chain [1] done processing
10:37:26 - cmdstanpy - INFO - Chain [1] start processing
10:37:26 - cmdstanpy - INFO - Chain [1] done processing
10:37:26 - cmdstanpy - INFO - Chain [1] start processing
10:37:26 - cmdstanpy - INFO - Chain [1] done processing
10:37:26 - cmdstanpy - INFO - Chain [1] start processing
10:37:26 - cmdstanpy - INFO - Chain [1] done processing
10:37:26 - cmdstanpy - INFO - Chain [1] start processing
10:37:27 - cmdstanpy - INFO - Chain [1] done processing
10:37:27 - cmdstanpy - INFO - Chain [1] start processing
10:37:27 - cmdstanpy - INFO - Chain [1]

,route_id,able_effect_pct,ace_incremental_effect_pct,total_effect_pct
0,B11,NaN,4.905607,4.905607
1,B25,-0.641714,1.859874,1.206224
2,B26,-0.350351,8.306860,7.927406
3,B35,NaN,-0.833366,-0.833366
4,B41,NaN,2.497470,2.497470
5,B44+,-1.433253,4.215916,2.722238
6,B46+,-2.276019,-2.732438,-4.946267
7,B60,NaN,7.250618,7.250618
8,B62,-6.087371,-6.349107,-12.049984
9,B63,NaN,-1.913931,-1.913931


In [110]:
import pandas as pd
import numpy as np

def robust_summary(series):
    s = series.dropna()
    return pd.Series({
        "n_routes": s.shape[0],
        "mean": s.mean(),
        "median": s.median(),
        "trimmed_mean_10pct": s.sort_values().iloc[int(0.1*len(s)) : int(0.9*len(s))].mean()
            if len(s) >= 10 else np.nan,
        "std_dev": s.std(),
        "mad": (s - s.median()).abs().median(),
        "min": s.min(),
        "q25": s.quantile(0.25),
        "q75": s.quantile(0.75),
        "max": s.max(),
        "pct_positive": (s > 0).mean() * 100,
        "pct_negative": (s < 0).mean() * 100,
        "pct_near_zero": (s.abs() < 0.5).mean() * 100
    })


summary_stats = pd.concat(
    {
        "ABLE effect (%)": robust_summary(df_effects["able_effect_pct"]),
        "ACE incremental effect (%)": robust_summary(df_effects["ace_incremental_effect_pct"]),
        "TOTAL compounded effect (%)": robust_summary(df_effects["total_effect_pct"])
    },
    axis=1
)

summary_stats


,ABLE effect (%),ACE incremental effect (%),TOTAL compounded effect (%)
n_routes,20.000000,53.000000,53.000000
mean,0.744825,1.861394,2.353297
median,-0.859939,1.113103,1.109695
trimmed_mean_10pct,-0.148750,1.220461,0.973167
std_dev,6.118135,7.259173,10.947881
mad,2.855279,3.351660,3.795912
min,-6.087371,-14.126111,-18.625489
q25,-2.491673,-1.813016,-1.913931
q75,3.024035,5.036596,5.245391
max,17.100420,31.304737,53.758399
